In [ ]:
import cv2
import mediapipe as mp
import time
import random
import numpy as np

class handDetector():
    def __init__(self, mode=False, maxHands=2, modelComplexity=1, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.modelComplexity = modelComplexity
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(static_image_mode=self.mode,
                                        max_num_hands=self.maxHands,
                                        model_complexity=self.modelComplexity,
                                        min_detection_confidence=self.detectionCon,
                                        min_tracking_confidence=self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
    
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
               if draw:
                self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

    def findPosition(self, img, handNo=0, draw=True):
        lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
        return lmList

class SnakeGame:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.snake = [(width // 2, height // 2)]
        self.direction = 'RIGHT'
        self.food = self.generate_food()
        self.score = 0

    def generate_food(self):
        while True:
            food = (random.randint(0, self.width - 1), random.randint(0, self.height - 1))
            if food not in self.snake:
                return food

    def move(self):
        head = self.snake[0]
        if self.direction == 'UP':
            new_head = (head[0], (head[1] - 1) % self.height)
        elif self.direction == 'DOWN':
            new_head = (head[0], (head[1] + 1) % self.height)
        elif self.direction == 'LEFT':
            new_head = ((head[0] - 1) % self.width, head[1])
        else:  # RIGHT
            new_head = ((head[0] + 1) % self.width, head[1])

        if new_head in self.snake:
            return False  # Game over

        self.snake.insert(0, new_head)

        if new_head == self.food:
            self.score += 1
            self.food = self.generate_food()
        else:
            self.snake.pop()

        return True

    def change_direction(self, new_direction):
        if (new_direction == 'UP' and self.direction != 'DOWN') or \
           (new_direction == 'DOWN' and self.direction != 'UP') or \
           (new_direction == 'LEFT' and self.direction != 'RIGHT') or \
           (new_direction == 'RIGHT' and self.direction != 'LEFT'):
            self.direction = new_direction

def main():
    cap = cv2.VideoCapture(0)
    detector = handDetector()
    game = SnakeGame(20, 15)
    cell_size = 20
    
    while True:
        success, img = cap.read()
        img = detector.findHands(img)
        lmList = detector.findPosition(img, draw=False)

        if len(lmList) != 0:
            # Use the position of the index finger (landmark 8) to control the snake
            x, y = lmList[8][1:]
            if x < img.shape[1] // 3:
                game.change_direction('RIGHT')
            elif x > 2 * img.shape[1] // 3:
                game.change_direction('LEFT')
            elif y < img.shape[0] // 2:
                game.change_direction('UP')
            else:
                game.change_direction('DOWN')

        if not game.move():
            print(f"Game Over! Score: {game.score}")
            break

        # Create a blank image for the game board
        board = np.zeros((game.height * cell_size, game.width * cell_size, 3), dtype=np.uint8)

        # Draw snake
        for segment in game.snake:
            cv2.rectangle(board, (segment[0] * cell_size, segment[1] * cell_size),
                          ((segment[0] + 1) * cell_size, (segment[1] + 1) * cell_size), (0, 255, 0), -1)

        # Draw food
        cv2.rectangle(board, (game.food[0] * cell_size, game.food[1] * cell_size),
                      ((game.food[0] + 1) * cell_size, (game.food[1] + 1) * cell_size), (0, 0, 255), -1)

        # Display score
        cv2.putText(img, f"Score: {game.score}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Display game board and camera feed
        cv2.imshow("Snake Game", board)
        cv2.imshow("Hand Tracking", img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

2024-08-07 12:53:06.456383: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1723053196.752394 3577036 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.8.16), renderer: Intel(R) Iris(TM) Graphics 6100
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1723053196.774372 3577825 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723053196.808779 3577828 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: 